<a href="https://colab.research.google.com/github/lmastalerz/sutainability/blob/main/FaaS_sustainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Is FaaS greener?**

Serverless and Function as a Service seem to be promising in terms of they sustainability, but are they really more energy effifcient?

Let's try to estimate.

I'll be focusing on Azure, but many of these concepts should apply to other FaaS providers. 
For the sake of simplicity I'm ignoring a ton of factors like memory, storage, network and focusing on **CPU energy usage** only. It seems to be the most important factor anyway, especially for the workloads that are good candidates for FaaS.

---

## Hardware
First, let's define energy usage of an average CPU core.
This data comes from [Cloud Carbon Footprint](https://www.cloudcarbonfootprint.org/docs/methodology) methodology page, which in turn follows [Etsy's Cloud Jewels](https://codeascraft.com/2020/04/23/cloud-jewels-estimating-kwh-in-the-cloud/)



In [169]:
AVG_MIN_WATTS = 0.78
AVG_MAX_WATTS = 3.76

## IaaS
Let's try to define baseline IaaS characteristics first.
It's not unheard of to run applications on, say, 4-core VM's.
They would also often run in pairs to ensure high availability and disaster recovery. At least this is likely to be what you end up moving your IaaS in a lift-and-shift approach from on-prem to the cloud (obviously you're not doing this and re-architecting everything as cloud-native, right?).

We need to make some assuptions about CPU utilization ono the VM's running the app. I'm taking 10% and something I've seen, but this parameter can be definitely tweaked.


In [170]:
# Number of cores VM gets
APP_VM_CORE_COUNT = 4

# Average CPU consumption of the VM running the app
APP_VM_USAGE_PER_CORE = 0.1

##FaaS
One of the most important FaaS characteristics in this context is how long the function stays "warm" after the execution. Let's assume 10 minutes, which is default for Azure Functions and some other FaaS frameworks.
FaaS cold start can be a complicated topic, details on what happens behin the scenes are available [here](https://azure.microsoft.com/en-us/blog/understanding-serverless-cold-start/). 

In [171]:
# How long function stays "warm"
FUN_TTL_MIN = 10

## Workload
Let's try to define characteristics of the workload we'd like to test now. 
We're assessing workload that could be a good candidate for FaaS and 
[Microsoft's Research](https://www.microsoft.com/en-us/research/uploads/prod/2020/05/serverless-ATC20.pdf) highlights some very interesting aspects of FaaS workloads.

For example, 90% of functions running as FaaS takes less than 60 seconds to executed, around 85% takes less than 10 seconds. Let's conservatively assume that our average function takes 10 seconds to run

In [172]:
# Average function execution time
AVG_FUN_DURATION_SEC = 10

According to the same [Microsoft's Research](https://www.microsoft.com/en-us/research/uploads/prod/2020/05/serverless-ATC20.pdf) invocation distribution of Azure Functions follows an interesting pattern. On a very high level, research shows that the distribution of invocation frequencies is heavily skewed towards functions executing very infrequently.  

In an attempt to simplify calculations, functions will be allocated to three buckets. 


In [173]:
# 40% of functions are being executed at least every 10 minutes.
# This implies, that with default time function stay's warm (10 min) the function 
# is never deallocated 
FUN_INVOCATION_BKT_1_SIZE = 0.4

# 40% of functions are being executed at most once every hour
# This implies that within every hour function is deallocated most of the time 
# (50 minutes with default 10 min TTL)
FUN_INVOCATION_BKT_2_SIZE = 0.4

# 20% of functions fall somewhere between these two extremens.
# They would execute more than once per hour but at the same time 
# sometimes will be deallocated because they didn't run for more than 10 mins
#
FUN_INVOCATION_BKT_3_SIZE = 0.2

Let's define number of apps for which we run the simulation.
For the sake of simplicity assumming there's no difference between Azure Function App and Azure Function (1:1 mapping). All distribution stats are taken for Function **App** with all its Functions inside, so this assumption shouldn't have an impact on results.  


In [174]:
# Number of apps
NUMBER_OF_APPS = 1000



---



### IaaS energy consumption
For the workload described by the characteristics given above, which should be a failrly representative FaaS-eligible workload, let's try to calculate energy consumption in two different scenarios to understand how much we're saving by running it as serverless. 

Let's start with IaaS.
We're running each app on it's own VM and we're deploying VM to two different data centers. 
Let's conservatively assume that all DR environments are only running as warm standby and don't to any processing under normal circumstances.

In [175]:
# Number of VM's in primary DC
NO_OF_PRIMARY_VMS = NO_OF_APPS
# Number of VM's in standby DC
NO_OF_STANDBY_VMS = NO_OF_APPS

# Number of cores used by primary servers
NO_OF_PRIMARY_CORES = NO_OF_PRIMARY_VMS * APP_VM_CORE_COUNT
# Number of cores used by standby servers
NO_OF_STANDBY_CORES = NO_OF_STANDBY_VMS * APP_VM_CORE_COUNT 

Now let's calculate energy used by these CPU's. 
Again, I'm ignoring energy footprint of memory, storage, network, etc.
I'm also assumming that standby environments are there but there's no processing happening there (they're running but CPU usage is at 0%)

In [176]:
# Energy used irrespective of whether anything is running
IAAS_IDLE_CPU_WATTS = NO_OF_PRIMARY_CORES * AVG_MIN_WATTS + NO_OF_STANDBY_CORES * AVG_MIN_WATTS

# Energy used by active processes (ignoring standby here)
IAAS_ACTIVE_CPU_WATTS = (AVG_MAX_WATTS - AVG_MIN_WATTS) * APP_VM_USAGE_PER_CORE * NO_OF_PRIMARY_CORES 

# Total energy used by the workload deployed on IaaS
IAAS_TOTAL_CPU_WATTS = IAAS_IDLE_CPU_WATTS + IAAS_ACTIVE_CPU_WATTS

In [177]:
print("Total energy consumption of", NO_OF_APPS, "is", IAAS_TOTAL_CPU_WATTS, "Watts")

Total energy consumption of 1000 is 7432.0 Watts


## FaaS energy consumption 
FaaS consumption would have to be divided into three distinct groups as per buckets defined before. 

###Bucket 1
Starting with functions that are being fired every 10 minutes of more often (bucket 1)

In [178]:
# Number of apps which are a part of first bucket
FAAS_BKT_1_NO_OF_APPS = NO_OF_APPS * FUN_INVOCATION_BKT_1_SIZE

These apps are never deallocated, so they behave similarly to IaaS.
Azure Functions have one core allocated to an instance (which can scale to another instances under high load - ignoring scalling for the sake of simplicity)

In [179]:
# Number of cores needed by apps in the first bucket is equivalent to the number of apps
FAAS_BKT_1_NO_CORES = FAAS_BKT_1_NO_OF_APPS

Based on the distirbution mantioned above, functions in this bucket would run on average every minute (average is an onversimplification here, but should be enough for the final calculation)

In [180]:
# Number of function invocations per hour for functions in bucket 1
FAAS_BKT_1_INVOCATIONS_PER_HOUR = 60

Let's calculate energy consumption of these CPU cores now

In [181]:
# Energy used irrespective of whether anything is running
FAAS_BKT_1_IDLE_CPU_WATTS = FAAS_BKT_1_NO_CORES * AVG_MIN_WATTS

# Energy used for actual code execution  
FAAS_BKT_1_ACTIVE_CPU_WATTS = (AVG_MAX_WATTS - AVG_MIN_WATTS) * FAAS_BKT_1_NO_CORES \
* FAAS_BKT_1_INVOCATIONS_PER_HOUR * AVG_FUN_DURATION_SEC / 3600 # Average CPU usage 

# Total energy used by the FaaS workload from bucket 1
FAAS_BKT_1_TOTAL_CPU_WATTS = FAAS_BKT_1_IDLE_CPU_WATTS + FAAS_BKT_1_ACTIVE_CPU_WATTS

FAAS_BKT_1_TOTAL_CPU_WATTS

510.66666666666663

###Bucket 2
Functions in this bucket fire at most once per hour


In [182]:
# Number of apps which are a part of second bucket
FAAS_BKT_2_NO_OF_APPS = NO_OF_APPS * FUN_INVOCATION_BKT_2_SIZE

These apps are only allocated for once execution per hour. 
With default 10 min TTL functions only need 1/6 of core to be allocated to them

In [183]:
# Number of cores needed by apps in the second bucket 
FAAS_BKT_2_NO_CORES = FAAS_BKT_2_NO_OF_APPS * FUN_TTL_MIN / 60

With a very conservative approach assumming that functions fire once per hour (in fact they run AT MOST once per hour)

In [184]:
# Number of function invocations per hour for functions in bucket 2
FAAS_BKT_2_INVOCATIONS_PER_HOUR = 1

And now calculating energy used by functions in this bucket

In [185]:
# Energy used irrespective of whether anything is running
FAAS_BKT_2_IDLE_CPU_WATTS = FAAS_BKT_2_NO_CORES * AVG_MIN_WATTS

# Energy used for actual code execution  
FAAS_BKT_2_ACTIVE_CPU_WATTS = (AVG_MAX_WATTS - AVG_MIN_WATTS) * FAAS_BKT_2_NO_CORES \
* FAAS_BKT_2_INVOCATIONS_PER_HOUR * AVG_FUN_DURATION_SEC / 3600 # Average CPU usage 

# Total energy used by the FaaS workload from bucket 1
FAAS_BKT_2_TOTAL_CPU_WATTS = FAAS_BKT_2_IDLE_CPU_WATTS + FAAS_BKT_2_ACTIVE_CPU_WATTS

FAAS_BKT_2_TOTAL_CPU_WATTS

52.55185185185186

###Bucket 3
Function in third (smallest) bucket execute a number of times per hour.

In [186]:
# Number of apps which are a part of third bucket
FAAS_BKT_3_NO_OF_APPS = NO_OF_APPS * FUN_INVOCATION_BKT_3_SIZE

For functions in this group assumming that they will be deallocated for part of the time which is implied by the invocation frequency lower than every 10 minutes and default 10 min TTL. 
Assumming that they are allocated 75% of the time. 

In [187]:
# Portion of the time when functions in this bucket are allocated
FAAS_BKT_3_ALLOC = 0.75

With this allocation number of cores needed by functions in this group can be calculated

In [188]:
# Number of cores needed by apps in the third bucket 
FAAS_BKT_3_NO_CORES = FAAS_BKT_3_NO_OF_APPS * FAAS_BKT_3_ALLOC

Number of executions per hour by definition needs to be greater than 1 and also allow for windows bigger than 10 minutes when functions are not running. Let's take 5. 
This seems very arbitrary, but given difference between energy used by idle cores and active cores in this bucket it makes pretty much no difference

In [189]:
# Number of function invocations per hour for functions in bucket 2
FAAS_BKT_3_INVOCATIONS_PER_HOUR = 5

Now we can calculate energy usage for functions in this bucket

In [190]:
# Energy used irrespective of whether anything is running
FAAS_BKT_3_IDLE_CPU_WATTS = FAAS_BKT_3_NO_CORES * AVG_MIN_WATTS


# Energy used for actual code execution  
FAAS_BKT_3_ACTIVE_CPU_WATTS = (AVG_MAX_WATTS - AVG_MIN_WATTS) * FAAS_BKT_3_NO_CORES \
* FAAS_BKT_3_INVOCATIONS_PER_HOUR * AVG_FUN_DURATION_SEC / 3600 # Average CPU usage 


# Total energy used by the FaaS workload from bucket 1
FAAS_BKT_3_TOTAL_CPU_WATTS = FAAS_BKT_3_IDLE_CPU_WATTS + FAAS_BKT_3_ACTIVE_CPU_WATTS

print("Idle", FAAS_BKT_3_IDLE_CPU_WATTS, "Active", FAAS_BKT_3_ACTIVE_CPU_WATTS, "Total", FAAS_BKT_3_TOTAL_CPU_WATTS)

Idle 117.0 Active 6.208333333333332 Total 123.20833333333333


## FaaS Total
Putting energy usage of functions from all FaaS buckets

In [191]:
# Total energy consumption of all FaaS executions 
FAAS_TOTAL_CPU_WATTS = FAAS_BKT_1_TOTAL_CPU_WATTS + FAAS_BKT_2_TOTAL_CPU_WATTS + FAAS_BKT_3_TOTAL_CPU_WATTS

# Final comparison 

In [192]:
print("Total energy used by IaaS deployments:", IAAS_TOTAL_CPU_WATTS)
print("Total energy used by FaaS deployments:", FAAS_TOTAL_CPU_WATTS)
print("Efficiency factor:", IAAS_TOTAL_CPU_WATTS / FAAS_TOTAL_CPU_WATTS)

Total energy used by IaaS deployments: 7432.0
Total energy used by FaaS deployments: 686.4268518518519
Efficiency factor: 10.82708227387936
